# Testing

In [25]:
library(data.table)

In [26]:
# read data files
p = fread("../models/MobHealthRecycling/output/testing/parameters-1-1.csv")
m = fread("../models/MobHealthRecycling/output/testing/mortality-1-1.csv")
c = fread("../models/MobHealthRecycling/output/testing/county-1-1.csv")
e = fread("../models/MobHealthRecycling/output/testing/environment-1-1.csv")
mi = fread("../models/MobHealthRecycling/output/testing/migration-1-1.csv")
ag = fread("../models/MobHealthRecycling/output/testing/agents-1-1.csv")

In [28]:
#mi

In [29]:
mi[id == 32931 & age < 19]

iteration,replicate,model_time,id,age,county,cz,time
<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>
1,1,259.6219,32931,10,17,27402,10.02410
1,1,259.8490,32931,10,21,13501,10.25116
1,1,261.2874,32931,11,6,28202,11.68963
1,1,262.1261,32931,12,21,13501,12.52827
1,1,262.4128,32931,12,8,26703,12.81494
1,1,263.5470,32931,13,29,3101,13.94915
1,1,264.3022,32931,14,13,30802,14.70443
1,1,265.0607,32931,15,23,9302,15.46289
1,1,267.1483,32931,17,10,36902,17.55044


In [31]:
t = ag[id == 32931, .(id, age, time_exposure)]

In [18]:
summary(c$model_time)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
     60      80     120     120     160     180 

In [15]:
summary(c$rank_slope)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-0.1267  0.1701  0.2359  0.2262  0.2960  0.4629 